In [ ]:
import pandas as pd
from IPython.display import Markdown, display
from ipywidgets import interact
from ipywidgets.widgets import (
    Dropdown, SelectionSlider, Checkbox
)
from datetime import datetime
import cufflinks as cf

## Getting the data 

In [ ]:
path_dict = dict(
    confirmed = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
                 "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"),
    death = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
             "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"),
    recovered = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
                 "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")
)

In [ ]:
def set_index(df):
    """Set the index for the data frame using the date
    
    Args:
        df: Pandas data frame obtained from John Hopkins repo
    """
    # Set region, country, lat and long as index
    index = pd.MultiIndex.from_frame(df.iloc[:, :4])
    # Set the index and transpose
    df = df.iloc[:, 4:].set_index(index).T
    # Set date as index
    return df.set_index(pd.to_datetime(df.index, dayfirst=False))

df_dict = {
    type_: pd.read_csv(path).pipe(set_index)
    for type_, path in path_dict.items()
}

In [ ]:
def get_data(df_input, 
             data_type, 
             output_path=None,
             label_total='<< Total >>', 
             plot_type='line'):
    
    # Make a copy of the data frame
    df_input = df_input.copy()
    
    # Add a new column with the total number per day
    df_input[(None, label_total, None, None)] = (
        df_input.groupby(level=1, axis=1)
        .sum().sum(axis=1)
    )

    df = (
        df_input
        .groupby(level=1, axis=1).sum()
    )
    
    max_str_len = len(label_total)
    df.columns = [c if len(c) <= max_str_len 
                  else f"{c[:max_str_len]}..." 
                  for c in df.columns]
    df_diff =  df - df.shift()
    
    # n = n_rows + 1 (for the total)
    n = df.shape[1]

    labels = [data_type[0].upper() + data_type[1:] + " (total)",
              data_type[0].upper() + data_type[1:] + " (per day)"]
    titles = [f"Number of {data_type} (total)",
              f"Number of {data_type} (per day)"]

    visibles = [[True] + (n-1) * ['legendonly'] + n * [False],
                 n * [False] + [True] + (n-1) * ['legendonly']]

    buttons = [dict(args=[{"visible": visible},
                          {"title": title, "yaxis_title": title}], 
                    label=label, method="update")
               for title, label, visible in zip(titles, labels, visibles)]

    fig_0 = df.iplot(kind='line', title=titles[0], yTitle=titles[0],
                     theme='ggplot', asFigure=True,
                     margin=dict(l=0, r=160, b=0, t=50, pad=0), online=True,
                     mode='lines+markers', rangeslider=True)

    fig_1 = df_diff.iplot(kind='bar', online=True, theme='ggplot', 
                          asFigure=True, rangeslider=True,
                          margin=dict(l=0, r=160, b=0, t=50, pad=0), )

    fig_0.add_traces(tuple(fig_1.select_traces()))    
    
    # Keep only the first selected in the legend
    [tr.update({'visible': v}) for tr, v in zip(fig_0.select_traces(), visibles[0])]

    fig_0.update_layout(updatemenus=[
        dict(type="buttons", direction="right", active=0,
             x=0.57, y=1.2, buttons=buttons)
    ])

    if output_path is not None:
        fig_0.write_html(output_path)
    
    return fig_0

In [ ]:
%%time
plot_info = (('confirmed', 'confirmed cases'),
             ('death', 'deaths'),
             ('recovered', 'recovered cases'))

for k, data_type in plot_info:
    fig = get_data(df_dict[k], data_type, f"plot_{k}.html")

<iframe id="igraph" scrolling="no" style="border:none;"
        seamless="seamless" src="plot.html" height="100%" width="100%">
</iframe>